In [ ]:
"""
#6.2 Tiktoken (04:19)

Model은 token 간의 통계적인 관계를 이해하도록 학습함
Model은 Limit 있어서, 원하는 텍스트들을 한번에 입력할 수 없음

#6.3 Vectors (11:56)

Embed은 사람이 읽는 텍스트를 컴퓨터가 이해할 수 있는 숫자들로 변환하는 작업
벡터 좀 더 정확히느 벡터화
문서마다각각의 벡터를 잡아줌.
우리는 단어에 3차원 벡터를 적용함. Openai 모델은 천 개 이상 차원 가능
1차원 Masculinity(남성성)
2차원 Feminity(여성성)
3차원 Royalty(왕족스러움
"""

In [1]:
from dotenv import load_dotenv

load_dotenv()  # .env 파일을 환경 변수로 로드

True

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings,CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

# chunk_size : 얼마나 큰 덩어리로 나눌지 결정할 수 있음
# chunk_overlap : 문장이나 문단을 분할할 때 앞 조각 일부분을 가져옴
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100
)

cache_dir = LocalFileStore("./.cache/")

loader = UnstructuredFileLoader("./files/현진건-운수_좋은_날.txt",encoding="utf-8")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

In [71]:
examples = [
    {
        "movie": "Top Gun",
        "emojis": "🛩️👨‍✈️🔥",
    },
    {
        "movie": "The Godfather",
        "emojis": "👨‍👨‍👦🔫🍝",
    },
    {
        "movie": "The Lion King",
        "emojis": "🦁👑🌍",
    },
    {
        "movie": "The Lord of the Rings",
        "emojis": "🧙‍♂️💍🔥",
    },
]

In [17]:
vectorstore.similarity_search("왜 죽었지")

[Document(page_content='치삼의 끄는 손을 뿌리치더니 김첨지는 눈물이 글썽글썽한 눈으로 싱그레\n웃는다.\n“죽기는 누가 죽어.”\n하고 득의가 양양.\n“죽기는 왜 죽어, 생때같이 살아만 있단다. 그 오라질 년이 밥을 죽이지.\n인제 나한테 속았다.”\n하고 어린애 모양으로 손뼉을 치며 웃는다.\n“이 사람이 정말 미쳤단 말인가. 나도 아주먼네가 앓는단 말은 들었는\n데.”\n하고 치삼이도 어느 불안을 느끼는 듯이 김첨지에게 또 돌아가라고 권하였\n다.\n“안 죽었어, 안 죽었대도 그래.”', metadata={'source': './files/현진건-운수_좋은_날.txt'}),
 Document(page_content='“우리 마누라가 죽었다네.”\n“뭐, 마누라가 죽다니, 언제?”\n“이놈아 언제는, 오늘이지.”\n“엣기 미친놈, 거짓말 말아.”\n“거짓말은 왜, 참말로 죽었어, 참말로…… 마누라 시체를 집에 뻐들쳐 놓\n고 내가 술을 먹다니, 내가 죽일 놈이야, 죽일 놈이야.”\n하고 김첨지는 엉엉 소리를 내어 운다.\n치삼은 흥이 조금 깨어지는 얼굴로,\n“원 이 사람이, 참말을 하나 거짓말을 하나. 그러면 집으로 가세, 가.”\n하고 우는 이의 팔을 잡아당기었다.\n치삼의 끄는 손을 뿌리치더니 김첨지는 눈물이 글썽글썽한 눈으로 싱그레\n웃는다.', metadata={'source': './files/현진건-운수_좋은_날.txt'}),
 Document(page_content='(세살먹이)에게 죽을 사줄 수도 있다 - 팔십 전을 손에 쥔 김 첨지의 마음\n은 푼푼하였다.\n그러나 그의 행운은 그걸로 그치지 않았다. 땀과 빗물이 섞여 흐르는 목덜\n미를 기름주머니가 다된 왜목 수건으로 닦으며, 그 학교 문을 돌아 나올 때\n였다. 뒤에서 “인력거!” 하고 부르는 소리가 난다. 자기를 불러 멈춘 사\n람이 그 학교 학생인 줄 김첨지는 한번 보고 짐작할 수 있었다. 그 학생은\n다짜고짜로,\n“남대문 정거장까지 얼마요.”\n라고 물었다. 

In [72]:
llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True, # 메세지 형태로 출력
)

In [73]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{movie}"),
        ("ai", "{emojis}"),
    ]
)

In [74]:
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [75]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "I'm an emoji master with creativity. Here's a movie expressed with 3 emojis."),
        example_prompt,
        ("human", "{movie}"),
    ]
)

In [80]:
def load_memory(_):
    return memory.load_memory_variables({})["history"]

In [81]:
chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

In [82]:
def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )
    print(result)

In [83]:
invoke_chain("Iron Man")

content='🔧🦸\u200d♂️💥'


In [84]:
invoke_chain("The Dark Knight")

content='🦇🃏🦹\u200d♂️'


In [85]:
invoke_chain("Interstellar")

content='🚀⏳🌌'
